# IV. Linear codes for Inner Product Masking

### $n=3$ shares, $\ell=8$ bits

Parameters:

- $Z=(X + L_2Y_2 + L_3Y_3, Y_2, Y_3)=X\mathbf{G} + Y\mathbf{H}$ where $X, Y=(Y_2, Y_3)$ and $Z$ are the sensitive variable, a mask and the protected variable, respectively. $\mathbf{G} = [1, 0, 0]$ and $\mathbf{H} = [[L_2, 1, 0], [L_3, 0, 1]]$ are two generator matrices of codes $\mathcal{C}$ and $\mathcal{D}$, resp.
- $L_2, L_3\in \mathbb{F}_{2^\ell}\backslash\{0\}$, thus there are 255*255=65025 linear codes for IPM. In order to save spaces, we take $L_2\leq L_3$ and $L_2=\alpha^i$ where $i\in \{0, 1, \ldots, 255\}$ (note that $\alpha^0=\alpha^{255}$ in $\mathbb{F}_{2^8}$). As a consequence, we have 32896 entries and one more for BKLC codes with parameter [24, 8, 8].
- Each element over $\mathbb{F}_{2^\ell}$ can be denoted as $\alpha^i$ where $i\in\{0, 1, \ldots, 254\}$
- $\mathbf{H}^\perp = [1, L_2, L_3]$ is the generator matirx of dual code $\mathcal{D}^\perp$

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd # Pandas for tables
from IPython.display import Latex
from IPython.display import HTML

In [2]:
def read_f(file_name):
    """Reading weight enumerators."""
    with open(file_name, 'r') as fp:
        wd = fp.read().split("]\n")[:-1] # "\n"
        wd = np.array([list(map(int, re.findall(r"\d+", elem))) for elem in wd])
        
    return wd

def find_ind(i):
    """Generating index of L_2 and L_3"""
    ii = 256*(256+1)/2-i-1
    kk = np.floor((np.sqrt(8*ii+1)-1)/2)
    irow = int(256-1-kk)
    icol = int(i-256*(256+1)/2 + (256-irow)*(257-irow)/2 +irow)
    
    return irow, icol

## 1. Loading all weight enumerators

In [3]:
wd = read_f("weight_distrib_n3k8.txt") # Weight distribution

# print(wd.shape) # 226 entries: 225 for IPM codes and one for BKLC codes

### 1.1 Generating values

In [13]:
alpha_all = np.array(['$\\alpha^{%d}$'%i for i in np.arange(256)])
d_all = np.zeros(len(wd))
B_all = np.zeros(len(wd))
L_2 = np.zeros(len(wd), dtype=int)
L_3 = np.zeros(len(wd), dtype=int)
for i in range(len(wd)):
    d_all[i] = wd[i][2]
    B_all[i] = wd[i][3]
    if i<len(wd)-1:
        irow, icol = find_ind(i)
        L_2[i] = irow
        L_3[i] = icol

### 1.2 Defining styles of dataframe

In [14]:
# Set properties for th, td and caption elements in dataframe
th_props = [('font-size', '14px'), ('text-align', 'left'), ('font-weight', 'bold'), ('background-color', '#E0E0E0')]
td_props = [('font-size', '13px'), ('text-align', 'left'), ('min-width', '80px')]
cp_props = [('font-size', '16px'), ('text-align', 'center')]
# Set table styles
styles = [dict(selector="th", props=th_props), dict(selector="td", props=td_props), dict(selector="caption", props=cp_props)]
cm_1 = sns.light_palette("red", as_cmap=True)
cm_2 = sns.light_palette("purple", as_cmap=True, reverse=True)

In [15]:
#df = pd.DataFrame({'$L_2$': alpha_all[L_2[:-1]], '$L_3$': alpha_all[L_3[:-1]], '$d_{\mathcal{D}}^\perp$': d_all[:-1], 
#                   '$B_{d_{\mathcal{D}}^\perp}$': B_all[:-1], 'Weight Enumerators': wd[:-1]})

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 400)


<span style="color:red"> **We omit the complete table I here to avoid wasting of spaces. Hereafter, we only show all codes with maximized $d_{\mathcal{D}}^\perp=8$.**</span>

## 2. Optimal linear codes for IPM

### 2.1 Linear codes with $d_{\mathcal{D}}^\perp=8$

We focus on the the linear codes with greater $d_{\mathcal{D}}^\perp$, which are better in the sense of side-channel resistance (from our paper).

In [16]:
# Finding the indices of d_C=8
d_index = []
d_index_L2 = []
d_index_L3 = []
d_C = 8
for i in range(len(wd)-1):
    if wd[i][2] == d_C:
        d_index.append(i)
        irow, icol = find_ind(i)
        d_index_L2.append(irow)
        d_index_L3.append(icol)

#d_index = np.array(d_index)

In [17]:
print(len(d_index))

114


In [18]:
def highlight(s, threshold, column):
    is_min = pd.Series(data=False, index=s.index)
    is_min[column] = (s.loc[column] <= threshold)
    return ['background-color: gold' if is_min.any() else '' for v in is_min]

In [19]:
df_4 = pd.DataFrame({'$L_2$': np.array(alpha_all)[d_index_L2], '$L_3$': np.array(alpha_all)[d_index_L3], '$d_{\mathcal{D}}^\perp$': 
                     d_all[d_index], '$B_{d_{\mathcal{D}}^\perp}$': B_all[d_index], 'Weight Enumerators': wd[d_index]})
df_4 = df_4.sort_values(by=['$B_{d_{\mathcal{D}}^\perp}$'], ascending=True)

(df_4.style
    .apply(highlight, threshold=7, column=['$B_{d_{\mathcal{D}}^\perp}$'], axis=1)
    .background_gradient(cmap=cm_2, subset=['$B_{d_{\mathcal{D}}^\perp}$' ])
    .set_caption('Tab. II Linear codes for IPM with $d_{\mathcal{D}}^\perp=8$.')
    .set_table_styles(styles))

,$L_2$,$L_3$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
19,$\alpha^{18}$,$\alpha^{183}$,8,7,"[0, 1, 8, 7, 9, 27, 10, 44, 11, 41, 12, 34, 13, 34, 14, 24, 15, 22, 16, 14, 17, 3, 18, 4, 19, 1]"
41,$\alpha^{72}$,$\alpha^{90}$,8,7,"[0, 1, 8, 7, 9, 27, 10, 44, 11, 41, 12, 34, 13, 34, 14, 24, 15, 22, 16, 14, 17, 3, 18, 4, 19, 1]"
105,$\alpha^{165}$,$\alpha^{237}$,8,7,"[0, 1, 8, 7, 9, 27, 10, 44, 11, 41, 12, 34, 13, 34, 14, 24, 15, 22, 16, 14, 17, 3, 18, 4, 19, 1]"
86,$\alpha^{112}$,$\alpha^{187}$,8,10,"[0, 1, 8, 10, 9, 30, 10, 33, 11, 37, 12, 44, 13, 36, 14, 22, 15, 18, 16, 17, 17, 6, 18, 1, 19, 1]"
34,$\alpha^{68}$,$\alpha^{180}$,8,10,"[0, 1, 8, 10, 9, 30, 10, 33, 11, 37, 12, 44, 13, 36, 14, 22, 15, 18, 16, 17, 17, 6, 18, 1, 19, 1]"
102,$\alpha^{164}$,$\alpha^{236}$,8,10,"[0, 1, 8, 10, 9, 20, 10, 49, 11, 41, 12, 36, 13, 28, 14, 22, 15, 30, 16, 9, 17, 8, 18, 1, 19, 1]"
20,$\alpha^{19}$,$\alpha^{183}$,8,10,"[0, 1, 8, 10, 9, 20, 10, 49, 11, 41, 12, 36, 13, 28, 14, 22, 15, 30, 16, 9, 17, 8, 18, 1, 19, 1]"
42,$\alpha^{72}$,$\alpha^{91}$,8,10,"[0, 1, 8, 10, 9, 20, 10, 49, 11, 41, 12, 36, 13, 28, 14, 22, 15, 30, 16, 9, 17, 8, 18, 1, 19, 1]"
51,$\alpha^{75}$,$\alpha^{143}$,8,10,"[0, 1, 8, 10, 9, 30, 10, 33, 11, 37, 12, 44, 13, 36, 14, 22, 15, 18, 16, 17, 17, 6, 18, 1, 19, 1]"
0,$\alpha^{7}$,$\alpha^{80}$,8,11,"[0, 1, 8, 11, 9, 32, 10, 28, 11, 36, 12, 44, 13, 34, 14, 34, 15, 18, 16, 8, 17, 6, 18, 2, 19, 2]"


### 2.2 Optimal codes for IPM

As shown in our paper, the codes satifying two conditions are optimal:

- Maximizing $d_{\mathcal{D}}^\perp$, here $\max\{d_{\mathcal{D}}^\perp\} = 8$
- Minimizing $B_{d_{\mathcal{D}}^\perp}$, here $\min\{B_{d_{\mathcal{D}}^\perp}\} = 7$

Note that we use two leakage detection metrics **SNR** (signal-to-noise ratio) and **MI** (mutual information), and one leakage exploitation metric **SR** (success rate) to assess the side-channel resistance of IPM with different codes.

As a result of Tab. II, we conclude that the optimal codes for IPM are genetated by $\mathbf{H}=[[L_2, 1, 0], [L_3, 0, 1]]$ where $(L_2, L_3)\in\{(\alpha^{18}, \alpha^{183}), (\alpha^{72}, \alpha^{90}), (\alpha^{165}, \alpha^{237})\}$. Note that by exchanging $L_2$ and $L_3$, the codes are still optimal (we omit the equivalent codes to save spaces here).

### 2.3 BKLC code with parameter $[24, 8, 8]$

BKLC is the short of Best Known Linear Code. Note that the code $[24, 8, 8]$ is unique.

In [20]:
bklc_index = [-1]
cm_3 = sns.light_palette("red", as_cmap=True, reverse=True)
df_bklc = pd.DataFrame({'$L_2$': ['  --'], '$L_3$': ['  --'], '$d_{\mathcal{D}}^\perp$': d_all[-1], '$B_{d_{\mathcal{D}}^\perp}$': 
                        B_all[-1], 'Weight Enumerators': wd[bklc_index]})

(df_bklc.style
    .background_gradient(cmap=cm_3, subset=['$d_{\mathcal{D}}^\perp$', '$B_{d_{\mathcal{D}}^\perp}$'])
    .set_caption('Tab. III A BKLC code for IPM with $d_{\mathcal{D}}^\perp=8$.')
    .set_table_styles(styles))

,$L_2$,$L_3$,$d_{\mathcal{D}}^\perp$,$B_{d_{\mathcal{D}}^\perp}$,Weight Enumerators
0,--,--,8,130,"[0, 1, 8, 130, 12, 120, 16, 5]"


We can see that this BKLC code is worse than all linear codes with $d_{\mathcal{D}}^\perp=8$ in IPM, since the $B_{d_{\mathcal{D}}^\perp}$ in this BKLC code is much greater than in IPM codes. It is interesting to notice that the BKLC code $[24, 8, 8]$ cannot be generated by any $\mathbf{H}^\perp=[1, L_2, L_3]$ where $L_2, L_3 \in \mathbb{F}_{2^8}\backslash\{0\}$.